# Vacation Rental Analysis and Revenue Forecasting

This project manages vacation rental data, integrates with a MySQL database, and provides a user-friendly interface for browsing, booking, and managing listings.

---

## Section 1: Importing Required Libraries

These libraries are essential for data manipulation, database interaction, and visualization.

In [1]:
import pandas as pd
from datetime import datetime
import random
import mysql.connector
from mysql.connector import Error
import matplotlib.pyplot as plt
import numpy as np

## Section 2: Loading Data from CSV Files

The following section loads data from CSV files into pandas DataFrames for processing.

In [4]:
properties_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/properties.csv")

In [6]:
users_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/users.csv")

In [8]:
amenities_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/amenities.csv")

In [10]:
property_amenities_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/property_amenities.csv")

In [12]:
owners_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/owners.csv")

In [22]:
reviews_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/reviews (1).csv")

In [24]:
user_feedback_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/feedback.csv")

In [26]:
requests_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/requests.csv")

In [28]:
transactions_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/transactions.csv")

In [30]:
cancellations_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/cancellations.csv")

In [32]:
bookings_df = pd.read_csv("/Users/anish/Downloads/Project IDMP/bookings.csv")

## Section 3: Database Integration

The following code defines a reusable function to insert data from a pandas DataFrame into MySQL tables.

In [35]:
def insert_dataframe(dataframe, table_name, database_config):
    """
    Inserts a pandas DataFrame into a MySQL table.
    """
    try:
        # Connect to the MySQL database
        conn = mysql.connector.connect(**database_config)
        cursor = conn.cursor()
        
        # Build the insert query
        columns = ', '.join(dataframe.columns)
        placeholders = ', '.join(['%s'] * len(dataframe.columns)) #(%s) used as placeholder
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        # Insert each row in the DataFrame
        for _, row in dataframe.iterrows():
            cursor.execute(query, tuple(row))
        
        # Commit the changes
        conn.commit()
        print(f"DataFrame inserted into the {table_name} table successfully.")
    
    except Error as e:
        print(f"Error: {e}")
    finally:
        # Close the connection
        if conn.is_connected():
            cursor.close()
            conn.close()

In [37]:
database_config = {
    "host": "localhost",
    "user": "root",
    "password": "$asukeUch1ha",
    "database": "vacation_rentals"
}

In [55]:
insert_dataframe(owners_df, "Owners", database_config)

DataFrame inserted into the Owners table successfully.


In [57]:
insert_dataframe(properties_df, "Properties", database_config)

DataFrame inserted into the Properties table successfully.


In [59]:
insert_dataframe(users_df, "Users", database_config)

DataFrame inserted into the Users table successfully.


In [135]:
insert_dataframe(amenities_df, "Amenities", database_config)

DataFrame inserted into the Amenities table successfully.


In [147]:
insert_dataframe(property_amenities_df, "Property_Amenities", database_config)

DataFrame inserted into the Property_Amenities table successfully.


In [209]:
insert_dataframe(reviews_df, "Reviews", database_config)

DataFrame inserted into the Reviews table successfully.


In [163]:
insert_dataframe(user_feedback_df, "User_Feedback", database_config)

DataFrame inserted into the User_Feedback table successfully.


In [224]:
insert_dataframe(requests_df, "Requests", database_config)

DataFrame inserted into the Requests table successfully.


In [185]:
insert_dataframe(bookings_df, "Bookings", database_config)

Error: 1062 (23000): Duplicate entry '71660' for key 'bookings.PRIMARY'


In [193]:
insert_dataframe(cancellations_df, "Cancellations", database_config)

DataFrame inserted into the Cancellations table successfully.


In [197]:
insert_dataframe(transactions_df, "Transactions", database_config)

DataFrame inserted into the Transactions table successfully.


In [189]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="$asukeUch1ha",
    database="vacation_rentals"
)

cursor = conn.cursor()

for _, row in bookings_df.iterrows():
    try:
        query = """
        INSERT IGNORE INTO Bookings (
            booking_id, property_id, user_id, check_in_date, check_out_date, nights_booked, total_price, payment_method
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(query, (
            row['booking_id'], 
            row['property_id'], 
            row['user_id'], 
            row['check_in_date'], 
            row['check_out_date'], 
            row['nights_booked'], 
            row['total_price'], 
            row['payment_method']
        ))
conn.commit()

### Creating connection to the MySQL Database.

In [39]:
def create_connection():
    """
    Function to connect to the MySQL Database
    """
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="$asukeUch1ha",
        database="vacation_rentals"
    )

conn = create_connection()
cursor = conn.cursor()

In [41]:
def ensure_connection():
    """
    Ensures the MySQL connection is active and reconnects if necessary.
    """
    global conn, cursor
    try:
        if not conn.is_connected():
            print("Reconnecting to the database...")
            conn = create_connection()
            cursor = conn.cursor()
    except mysql.connector.Error as e:
        print(f"Error while reconnecting to the database: {e}")
        exit(1)

In [43]:
def execute_query(query, params=None):
    """
    Executes a query with optional parameters and ensures connection is active.
    """
    try:
        ensure_connection()
        cursor.execute(query, params)
    except mysql.connector.Error as e:
        print(f"Error executing query: {e}")
        ensure_connection()
        cursor.execute(query, params)

### Defining the Main Menu for login

In [46]:
def main_menu():
    """
    Main menu for the application.
    """
    while True:
        print("\nWelcome to the Vacation Rental System")
        print("1. Owner Login")
        print("2. User Login")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            owner_menu()
        elif choice == '2':
            user_menu()
        elif choice == '3':
            print("Goodbye!")
            cursor.close()
            conn.close()
            break
        else:
            print("Invalid choice. Please try again.")

## Section 4: User Interface

This section builds the main menu for users and provides options to log in, browse, and filter listings.

In [49]:
def user_login():
    """
    Handles user login or account creation.
    """
    ensure_connection()

    print("\nUser Login")
    print("1. Log in with an existing User ID")
    print("2. Create a new account")
    choice = input("Enter your choice (1/2): ").strip()

    if choice == '1':
        user_id = input("Enter your User ID: ").strip()

        try:
            query = "SELECT user_name FROM users WHERE user_id = %s" #Fetch the user_name and user_id from the Users table
            execute_query(query, (user_id,))
            result = cursor.fetchone()

            if result:
                user_name = result[0]
                print(f"Welcome back, {user_name}! Your user ID is {user_id}.")
                return int(user_id)
            else:
                print("User ID not found. Please try again.")
                return user_login()
        except mysql.connector.Error as e:
            print(f"Error during login: {e}")

    elif choice == '2':
        return create_new_account()

    else:
        print("Invalid choice. Please try again.")
        return user_login()

### New account creation

In [52]:
def create_new_account():
    """
    Creates a new user account with a unique user_id, phone number, and default user rating.
    """
    ensure_connection()

    while True:
        # Generate a unique user_id
        user_id = random.randint(10000, 99999)
        query = "SELECT COUNT(*) FROM users WHERE user_id = %s"
        execute_query(query, (user_id,))
        count = cursor.fetchone()[0] #Make sure that the user_id doesn't repeat

        if count == 0:  # If the user_id is unique, proceed
            break

    user_name = input("Enter your name to create a new account: ").strip()

    while True:
        phone_number = input("Enter your phone number (digits only): ").strip()
        if phone_number.isdigit() and len(phone_number) >= 10:
            break
        else:
            print("Invalid phone number. Please enter a valid phone number.")

    # Default user rating to 3.0
    user_rating = 3.0

    try:
        # Insert the new user into the database
        query = """
            INSERT INTO users (user_id, user_name, phone_number, user_rating)
            VALUES (%s, %s, %s, %s)
        """
        execute_query(query, (user_id, user_name, phone_number, user_rating))
        conn.commit()

        print(f"Welcome, {user_name}! Your new user ID is {user_id}.")
        return user_id
    except mysql.connector.Error as e:
        print(f"Error: {e}")

### User Menu

In [55]:
def user_menu():
    """
    Menu for user functionalities.
    """
    user_id = user_login()
    while True:
        print("\nUser Menu")
        print(f"Logged in as User ID: {user_id}")
        print("1. Browse Listings")
        print("2. Filter Listings")
        print("3. Leave a Review")
        print("4. View Property Reviews")
        print("5. Logout")
        choice = input("Enter your choice: ")

        if choice == '1':
            browse_listings()
        elif choice == '2':
            filter_listings(user_id)
        elif choice == '3':
            leave_review(user_id)
        elif choice == '4':
            view_reviews()
        elif choice == '5':
            print("Logging out... Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

### Browse all listings

In [58]:
def browse_listings():
    """
    View all listings on the platform, showing availability based on bookings.
    """
    ensure_connection()

    # SQL query to check availability based on bookings
    query = """
        SELECT 
            p.property_id, 
            p.name, 
            p.location,
            p.room_type, 
            p.listing_price,
            CASE 
                WHEN EXISTS (
                    SELECT 1 
                    FROM bookings b 
                    WHERE b.property_id = p.property_id 
                      AND CURDATE() BETWEEN b.check_in_date AND b.check_out_date
                ) THEN 'Not Available'
                ELSE 'Available Right Now!'
            END AS availability_status
        FROM Properties AS p
    """
    
    try:
        execute_query(query)
        results = cursor.fetchall()

        if results:
            print("\nAvailable Listings:")
            print(f"{'Property ID':<7} {'Name':<30} {'Location':<10} {'Room Type':<10} {'Listing Price':<5} {'Status':<15}")
            print("-" * 95)

            # Display each listing
            for row in results:
                property_id, name, location, room_type, listing_price, status = row
                print(f"{property_id:<12} {name:<20} {location:<15} {room_type:<15} ${listing_price:<14.2f} {status:<10}")
        else:
            print("\nNo listings available at the moment.")
    except mysql.connector.Error as e:
        print(f"Error fetching listings: {e}")

### Filter Listings Based on Price and Location

In [61]:
def filter_listings(user_id):
    """
    Filters listings based on maximum price and location.
    Displays availability information and allows booking directly.
    """
    try:
        # Get user input for filters
        max_price = float(input("Enter the maximum price per night: "))
        location = input("What is your desired location (leave blank for all locations): ").strip()

        # Fetch filtered listings
        listings = fetch_filtered_listings(max_price, location)

        if not listings:
            print("Oops! No listings match the specified criteria.")
            return

        # Display filtered listings with availability
        print("\nFiltered Listings:")
        for idx, (property_id, name, price, loc) in enumerate(listings, start=1):
            # Fetch availability information
            availability_info = fetch_availability_info(property_id)
            
            # Display listing details
            print(f"{idx}. Property ID: {property_id}, Name: {name or 'No name'}, "
                  f"Price: ${price:.2f}, Location: {loc or 'No location'}")
            print(f"   Availability: {availability_info}")
            print("-" * 50)

        # Allow user to select a property to book
        selection = get_user_selection(len(listings))
        if selection == 0:
            print("Exiting to main menu.")
            return

        # Process booking
        selected_property = listings[selection - 1]
        handle_booking(user_id, selected_property) 

    except mysql.connector.Error as e:
        print(f"Database error: {e}")
    except ValueError:
        print("Invalid input. Please enter valid data.")

In [63]:
def fetch_filtered_listings(max_price, location):
    """Fetches filtered listings based on price and location."""
    query = """
        SELECT DISTINCT p.property_id, p.name, p.listing_price, p.location
        FROM properties p
        WHERE p.listing_price <= %s
    """
    params = [max_price]

    if location:
        query += " AND p.location = %s"
        params.append(location)

    execute_query(query, tuple(params))
    return cursor.fetchall()

In [65]:
def display_listings(listings):
    """Displays the listings to the user."""
    print("\nFiltered Listings:")
    for idx, (property_id, name, price, loc) in enumerate(listings, start=1):
        print(f"{idx}. Property ID: {property_id}, Name: {name or 'No name'}, "
              f"Price: ${price:.2f}, Location: {loc or 'No location'}")

In [67]:
def get_user_selection(num_listings):
    """Gets the user's selection for a listing."""
    while True:
        try:
            selection = int(input("\nEnter the number of the property you want to book, or 0 to exit: "))
            if 0 <= selection <= num_listings:
                return selection
            print(f"Invalid input. Please select a number between 0 and {num_listings}.")
        except ValueError:
            print("Invalid input. Please enter a number.")

In [69]:
def handle_booking(user_id, selected_property):
    """Handles booking for the selected property."""
    property_id, name, price, loc = selected_property

    check_in_date = input("Enter Check-in Date (YYYY-MM-DD): ").strip()
    check_out_date = input("Enter Check-out Date (YYYY-MM-DD): ").strip()

    # Validate booking dates
    check_in, check_out = validate_dates(check_in_date, check_out_date)
    if not check_in or not check_out:
        return

    # Check availability for the selected dates
    if not is_property_available(property_id, check_in_date, check_out_date):
        print("We're sorry. The property is not available for the selected date range.")
        return

    # Get the owner of the property
    owner_id = get_owner_id(property_id)
    if not owner_id:
        print("Property not found.")
        return

    # Create the booking request (request_id is auto-incremented by the database)
    create_booking_request(user_id, property_id, owner_id, check_in_date, check_out_date)

In [71]:
def validate_dates(check_in_date, check_out_date):
    """Validates the check-in and check-out dates."""
    try:
        check_in = datetime.strptime(check_in_date, "%Y-%m-%d")
        check_out = datetime.strptime(check_out_date, "%Y-%m-%d")
        if check_in >= check_out:
            print("Check-out date must be after check-in date.")
            return None, None
        return check_in, check_out
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return None, None

In [73]:
def is_property_available(property_id, check_in_date, check_out_date):
    """Checks if a property is available for the specified date range."""
    query = """
        SELECT COUNT(*)
        FROM bookings
        WHERE property_id = %s
        AND (check_in_date < %s AND check_out_date > %s)
    """
    execute_query(query, (property_id, check_out_date, check_in_date))
    return cursor.fetchone()[0] == 0

In [75]:
def get_owner_id(property_id):
    """Fetches the owner ID for a given property."""
    query = "SELECT owner_id FROM properties WHERE property_id = %s"
    execute_query(query, (property_id,))
    result = cursor.fetchone()
    return result[0] if result else None

In [77]:
def create_booking_request(user_id, property_id, owner_id, check_in_date, check_out_date):
    """
    Inserts a booking request into the requests table.
    MySQL auto-generates the request_id as it is an AUTO_INCREMENT column.
    """
    query = """
        INSERT INTO requests (user_id, property_id, owner_id, check_in_date, check_out_date, status)
        VALUES (%s, %s, %s, %s, %s, 'Pending')
    """
    execute_query(query, (user_id, property_id, owner_id, check_in_date, check_out_date))
    conn.commit()

    # Fetch the auto-generated request_id
    request_id = cursor.lastrowid
    print(f"Booking request sent successfully. Request ID: {request_id}")

In [79]:
def fetch_availability_info(property_id):
    """
    Fetches availability information for a property from the bookings table.
    Returns a formatted string with availability dates or a message if always available.
    """
    query = """
        SELECT check_in_date, check_out_date
        FROM bookings
        WHERE property_id = %s
        ORDER BY check_in_date
    """
    execute_query(query, (property_id,))
    results = cursor.fetchall()

    if not results:
        return "Always Available"

    # Format availability information
    availability = [f"Unavailable from {row[0]} to {row[1]}" for row in results]
    return "; ".join(availability)

### View Property Reviews

In [82]:
def view_reviews():
    """
    Allows the user to view reviews for a specific property.
    """
    try:
        # Prompt user for a property ID
        property_id = input("Enter the Property ID to view reviews: ").strip()

        # Fetch reviews for the property
        query = """
            SELECT r.review_id, r.property_id, u.user_name, r.review_date, r.comments,
                   r.rating_cleanliness, r.rating_host, r.rating_location, r.rating_overall
            FROM reviews r
            INNER JOIN users u ON r.user_id = u.user_id
            WHERE r.property_id = %s
            ORDER BY r.review_date DESC;
        """
        reviews_df = pd.read_sql(query, conn, params=(property_id,))

        # Check if there are any reviews
        if reviews_df.empty:
            print(f"No reviews found for Property ID {property_id}.")
            return

        # Display reviews
        print(f"\nReviews for Property ID {property_id}:")
        for _, row in reviews_df.iterrows():
            print(f"\nReview ID: {row['review_id']}")
            print(f"User: {row['user_name']} | Date: {row['review_date']}")
            print(f"Cleanliness: {row['rating_cleanliness']} | Host: {row['rating_host']} "
                  f"| Location: {row['rating_location']} | Overall: {row['rating_overall']}")
            print(f"Comments: {row['comments']}\n{'-' * 40}")
    except Exception as e:
        print(f"Error fetching reviews: {e}")

### Reviewing the Property

In [85]:
def leave_review(user_id):
    """
    Allows a user to leave a review for a property they've visited.
    """
    try:
        # Step 1: The query joins bookings and properties too fetch the properties that the user has visited.
        query = """
            SELECT DISTINCT b.property_id, p.name
            FROM bookings b
            JOIN properties p ON b.property_id = p.property_id
            WHERE b.user_id = %s
        """
        execute_query(query, (user_id,))
        visited_properties = cursor.fetchall()

        if not visited_properties:
            print("It seems you haven’t visited any properties yet. You need to book a stay before leaving a review.")
            return

        print("\nHere are the properties you've visited:")
        for idx, (property_id, name) in enumerate(visited_properties, start=1):
            print(f"{idx}. {name or 'Unnamed Property'} (Property ID: {property_id})")

        # Step 2: User selects a property to review
        while True:
            try:
                selection = int(input("Enter the number of the property you'd like to review (or 0 to exit): "))
                if selection == 0:
                    print("No worries! Come back when you're ready to leave a review.")
                    return
                if 1 <= selection <= len(visited_properties):
                    property_id = visited_properties[selection - 1][0] #Adjusting for zero index
                    break
                print("Invalid selection. Please choose a valid number.")
            except ValueError:
                print("Please enter a valid number.")

        # Step 3: Collect review details
        review_date = input("When did you visit this property? (YYYY-MM-DD): ").strip()
        try:
            datetime.strptime(review_date, "%Y-%m-%d")
        except ValueError:
            print("Invalid date format. Use YYYY-MM-DD.")
            return

        comments = input("How was your stay? (Feel free to share your thoughts): ").strip()

        print("\nLet’s rate your experience!")
        cleanliness = get_rating("Cleanliness")
        host = get_rating("Host")
        location = get_rating("Location")

        # Calculate overall rating
        overall_rating = round((cleanliness + host + location) / 3, 2)

        # Step 4: Insert the review into the database
        query = """
            INSERT INTO reviews (
                property_id, user_id, review_date, comments, rating_cleanliness,
                rating_host, rating_location, rating_overall
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        execute_query(query, (
            property_id, user_id, review_date, comments,
            cleanliness, host, location, overall_rating
        ))
        conn.commit()

        review_id = cursor.lastrowid
        print(f"\nThank you for your feedback! Your review has been successfully submitted.")
    except mysql.connector.Error as e:
        print(f"Oops, there was an issue submitting your review: {e}")
    except Exception as e:
        print(f"Something went wrong: {e}")

In [87]:
def get_rating(category):
    """
    Prompt the user to provide a rating between 0 and 5.
    """
    while True:
        try:
            rating = float(input(f"Rate {category} (0-5): "))
            if 0 <= rating <= 5:
                return rating
            print("Please enter a rating between 0 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number.")

## Section 5: Owner Interface

Allows property owners to create, update, and delete listings, as well as manage booking requests.

### Owner Login

In [91]:
def owner_login():
    """
    Handles owner login or account creation.
    """
    ensure_connection()

    print("\nOwner Login")
    print("1. Log in with an existing Owner ID")
    print("2. Create a new account")
    choice = input("Enter your choice (1/2): ").strip()

    if choice == '1':
        # Log in with existing Owner ID
        owner_id = input("Enter your Owner ID: ").strip()

        try:
            query = "SELECT owner_name FROM owners WHERE owner_id = %s"
            execute_query(query, (owner_id,))
            result = cursor.fetchone()

            if result:
                owner_name = result[0]
                print(f"Welcome back, {owner_name}! Your owner ID is {owner_id}.")
                return int(owner_id)
            else:
                print("Owner ID not found. Please try again.")
                return owner_login()
        except mysql.connector.Error as e:
            print(f"Error during login: {e}")

    elif choice == '2':
        # Create a new account
        return create_new_owner_account()

    else:
        print("Invalid choice. Please try again.")
        return owner_login()

## New account creation

In [94]:
def create_new_owner_account():
    """
    Creates a new owner account with an email address as contact information.
    """
    ensure_connection()

    while True:
        owner_id = random.randint(1000, 99999)
        # Check if the generated owner_id already exists in the database
        query = "SELECT COUNT(*) FROM owners WHERE owner_id = %s"
        execute_query(query, (owner_id,))
        count = cursor.fetchone()[0]

        if count == 0:  # If the ID is unique, proceed
            break

    owner_name = input("Enter your name to create a new account: ").strip()
    contact_info = input("Enter your email address: ").strip()

    try:
        #This query inserts the owners details provided in the owners table
        query = """
            INSERT INTO owners (owner_id, owner_name, contact_info)
            VALUES (%s, %s, %s)
        """
        execute_query(query, (owner_id, owner_name, contact_info))
        conn.commit()
        print(f"Welcome, {owner_name}! Your new owner ID is {owner_id}.")
        return owner_id
    except mysql.connector.Error as e:
        print(f"Error: {e}")

## Owner Menu

In [97]:
def owner_menu():
    """
    Menu for owner functionalities.
    """
    owner_id = owner_login()  # Login or create an account
    while True:
        print("\nOwner Menu")
        print(f"Logged in as Owner ID: {owner_id}")
        print("1. Create New Listing")
        print("2. Update Listing")
        print("3. Delete Listing")
        print("4. Manage Booking Requests")
        print("5. Logout")
        choice = input("Enter your choice: ")

        if choice == '1':
            create_listing(owner_id)
        elif choice == '2':
            update_listing(owner_id)
        elif choice == '3':
            delete_listing(owner_id)
        elif choice == '4':
            view_and_manage_requests(owner_id)
        elif choice == '5':
            print("Logging out...")
            break
        else:
            print("Invalid choice. Please try again.")

## Create New Listing

In [100]:
def create_listing(owner_id):
    """
    Creates a new listing for the owner and associates amenities from a separate table.
    """
    try:
        # Generate a unique property ID
        while True:
            property_id = random.randint(10000, 99999)
            query = "SELECT COUNT(*) FROM properties WHERE property_id = %s"
            execute_query(query, (property_id,))
            count = cursor.fetchone()[0]
            if count == 0:  # Ensure property_id is unique
                break

        # Collect property details
        name = input("Enter Property Name: ")
        location = input("Enter Location: ")
        room_type = input("Enter Room Type: ")
        listing_price = float(input("Enter Listing Price: "))
        occupancy_rate = 0.0  # Default occupancy rate

        # Insert property details into the properties table
        property_query = """
            INSERT INTO properties (property_id, name, owner_id, location, room_type, listing_price, occupancy_rate)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        execute_query(property_query, (property_id, name, owner_id, location, room_type, listing_price, occupancy_rate))
        conn.commit()
        print(f"Property '{name}' created successfully with Property ID: {property_id}.")

        # Associate amenities (this part remains the same)
        print("\nNow, let's add amenities to your property.")
        existing_amenities = fetch_existing_amenities()

        if existing_amenities:
            print("\nExisting Amenities:")
            for idx, amenity in enumerate(existing_amenities, start=1):
                print(f"{idx}. {amenity[1]} (Amenity ID: {amenity[0]})")
        else:
            print("\nNo existing amenities found. You will need to add new ones.")

        selected_amenities = select_amenities(existing_amenities)
        insert_property_amenities(property_id, selected_amenities)

    except mysql.connector.Error as e:
        print(f"Error: {e}")
    except ValueError:
        print("Invalid input. Please try again.")

In [102]:
def fetch_existing_amenities():
    """Fetches all existing amenities from the amenities table."""
    query = "SELECT amenity_id, amenity_name FROM amenities"
    execute_query(query)
    return cursor.fetchall()

In [104]:
def select_amenities(existing_amenities):
    """Prompts the user to select amenities or add new ones."""
    selected_amenities = []

    # Allow user to select from existing amenities
    if existing_amenities:
        while True:
            selection = input("\nEnter the number of an amenity to add (or 'done' to finish): ").strip()
            if selection.lower() == "done":
                break
            try:
                idx = int(selection) - 1
                if 0 <= idx < len(existing_amenities):
                    selected_amenities.append(existing_amenities[idx][0])  # Add amenity_id
                    print(f"Amenity '{existing_amenities[idx][1]}' added.")
                else:
                    print("Invalid selection. Please choose a valid number.")
            except ValueError:
                print("Invalid input. Please enter a number or 'done'.")

    # Allow user to add new amenities
    while True:
        new_amenity = input("\nEnter a new amenity to add (or 'done' to finish): ").strip()
        if new_amenity.lower() == "done":
            break
        amenity_id = add_new_amenity(new_amenity)
        selected_amenities.append(amenity_id)
        print(f"New amenity '{new_amenity}' added with Amenity ID: {amenity_id}.")

    return selected_amenities

In [106]:
def add_new_amenity(amenity_name):
    """Inserts a new amenity into the amenities table and returns its ID."""
    query = "INSERT INTO amenities (amenity_name) VALUES (%s)"
    execute_query(query, (amenity_name,))
    conn.commit()
    return cursor.lastrowid  # Fetch the auto-generated amenity_id

In [108]:
def insert_property_amenities(property_id, amenity_ids):
    """Links the property with the selected amenities."""
    query = "INSERT INTO property_amenities (property_id, amenity_id) VALUES (%s, %s)"
    for amenity_id in amenity_ids:
        execute_query(query, (property_id, amenity_id))
    conn.commit()
    print(f"Amenities successfully linked to Property ID: {property_id}.")

## Update Listing

In [111]:
def update_listing(owner_id):
    """
    Displays all properties owned by the owner and allows updates to specific fields.
    Excludes amenities from the update options.
    """
    try:
        # Step 1: Fetch and display all properties owned by the owner
        query = "SELECT property_id, name, location, room_type, listing_price FROM properties WHERE owner_id = %s"
        execute_query(query, (owner_id,))
        properties = cursor.fetchall()

        if not properties:
            print("You do not own any properties. Please add a property first.")
            return

        print("\nYour Properties:")
        for idx, (property_id, name, location, room_type, listing_price) in enumerate(properties, start=1):
            print(f"{idx}. Property ID: {property_id}, Name: {name}, Location: {location}, "
                  f"Room Type: {room_type}, Listing Price: ${listing_price:.2f}")

        # Step 2: Allow the owner to select a property to update
        try:
            selection = int(input("\nEnter the number of the property you want to update (or 0 to cancel): "))
            if selection == 0:
                print("No updates made. Exiting.")
                return

            if 1 <= selection <= len(properties):
                property_id = properties[selection - 1][0]  # Get the selected property's ID
            else:
                print("Invalid selection. Please choose a valid property number.")
                return
        except ValueError:
            print("Invalid input. Please enter a number.")
            return

        # Step 3: Allow the owner to update specific fields
        valid_columns = ["name", "location", "room_type", "listing_price"]
        print("\nYou can update the following fields:", ", ".join(valid_columns))

        column = input("Enter the column you want to update: ").strip()
        if column not in valid_columns:
            print(f"Invalid column. You can only update: {', '.join(valid_columns)}")
            return

        new_value = input(f"Enter the new value for {column}: ").strip()

        # Convert new_value to the appropriate type for listing_price
        if column == "listing_price":
            try:
                new_value = float(new_value)
            except ValueError:
                print("Invalid value. Listing price must be a number.")
                return

        # Update the property
        query = f"UPDATE properties SET {column} = %s WHERE property_id = %s"
        execute_query(query, (new_value, property_id))
        conn.commit()
        print(f"Property ID {property_id} updated successfully.")
    except mysql.connector.Error as e:
        print(f"Database error: {e}")
    except ValueError:
        print("Invalid input. Please try again.")

## Delete listing

In [114]:
def delete_listing(owner_id):
    """
    Deletes a listing owned by the owner and removes related entries from property_amenities.
    """
    try:
        # Step 1: Fetch and display all properties owned by the owner
        query = "SELECT property_id, name, location, room_type, listing_price FROM properties WHERE owner_id = %s"
        execute_query(query, (owner_id,))
        properties = cursor.fetchall()

        if not properties:
            print("You do not own any properties to delete.")
            return

        print("\nYour Properties:")
        for idx, (property_id, name, location, room_type, listing_price) in enumerate(properties, start=1):
            print(f"{idx}. Property ID: {property_id}, Name: {name}, Location: {location}, "
                  f"Room Type: {room_type}, Listing Price: ${listing_price:.2f}")

        # Step 2: Allow the owner to select a property to delete
        try:
            selection = int(input("\nEnter the number of the property you want to delete (or 0 to cancel): "))
            if selection == 0:
                print("No properties deleted. Exiting.")
                return

            if 1 <= selection <= len(properties):
                property_id = properties[selection - 1][0]  # Get the selected property's ID
            else:
                print("Invalid selection. Please choose a valid property number.")
                return
        except ValueError:
            print("Invalid input. Please enter a number.")
            return

        # Step 3: Delete related entries from property_amenities
        delete_amenities_query = "DELETE FROM property_amenities WHERE property_id = %s"
        execute_query(delete_amenities_query, (property_id,))
        conn.commit()
        print(f"Related amenities for Property ID {property_id} removed successfully.")

        # Step 4: Delete the property itself
        delete_property_query = "DELETE FROM properties WHERE property_id = %s"
        execute_query(delete_property_query, (property_id,))
        conn.commit()
        print(f"Property ID {property_id} deleted successfully.")
    except mysql.connector.Error as e:
        print(f"Database error: {e}")
    except ValueError:
        print("Invalid input. Please try again.")

## View and Manage Requests

In [117]:
def view_and_manage_requests(owner_id):
    """
    Allows the owner to view and respond to booking requests for their properties.
    Owners can accept or deny requests. Accepted requests are added to the bookings table.
    """
    try:
        # Fetch and display pending requests
        requests = fetch_pending_requests(owner_id)
        if not requests:
            print("You have no pending booking requests for your properties.")
            return

        display_pending_requests(requests)

        while True:
            # Ask the owner to respond to a specific request
            request_id = input("\nEnter the Request ID to manage (or type 'exit' to go back): ").strip()
            if request_id.lower() == 'exit':
                print("Exiting request management.")
                break

            try:
                request_id = int(request_id)
            except ValueError:
                print("Invalid input. Please enter a numeric Request ID.")
                continue

            # Validate the request and manage it
            if validate_request(request_id, owner_id):
                handle_request_action(request_id, owner_id)
            else:
                print("Invalid Request ID or the request is no longer pending.")
    except Exception as e:
        print(f"Error: {e}")

In [119]:
def fetch_pending_requests(owner_id):
    """
    Fetches all pending requests for the given owner.
    """
    query = """
        SELECT r.request_id, r.property_id, r.check_in_date, r.check_out_date, r.status, 
               u.user_name, p.listing_price
        FROM requests r
        INNER JOIN users u ON r.user_id = u.user_id
        INNER JOIN properties p ON r.property_id = p.property_id
        WHERE r.owner_id = %s AND r.status = 'Pending'
    """
    execute_query(query, (owner_id,))
    return cursor.fetchall()

In [121]:
def display_pending_requests(requests):
    """
    Displays a list of pending requests in a user-friendly format.
    """
    print("\nHere are your pending booking requests:")
    for idx, req in enumerate(requests, start=1):
        request_id, property_id, check_in_date, check_out_date, status, user_name, listing_price = req
        nights_requested = (check_out_date - check_in_date).days
        print(f"{idx}. Request ID: {request_id}, Property ID: {property_id}, "
              f"Check-in: {check_in_date}, Check-out: {check_out_date}, "
              f"Nights: {nights_requested}, Guest: {user_name}, "
              f"Price per Night: ${listing_price:.2f}, Status: {status}")

In [123]:
def validate_request(request_id, owner_id):
    """
    Validates that the request exists, belongs to the owner, and is pending.
    """
    query = """
        SELECT request_id, property_id, check_in_date, check_out_date
        FROM requests
        WHERE request_id = %s AND owner_id = %s AND status = 'Pending'
    """
    execute_query(query, (request_id, owner_id))
    return cursor.fetchone()

In [125]:
def handle_request_action(request_id, owner_id):
    """
    Handles accepting or denying a booking request.
    """
    decision = input(f"\nDo you want to Accept or Deny Request ID {request_id}? (A/D): ").strip().upper()
    if decision == "A":
        accept_request(request_id)
    elif decision == "D":
        deny_request(request_id)
    else:
        print("Invalid input. Please enter 'A' to Accept or 'D' to Deny.")

In [127]:
def accept_request(request_id):
    """
    Accepts a booking request, adds it to the bookings table, and updates the request with the generated booking_id.
    """
    try:
        # Mark the request as accepted
        query = "UPDATE requests SET status = 'Accepted' WHERE request_id = %s"
        execute_query(query, (request_id,))
        conn.commit()
        print(f"Request ID {request_id} has been accepted.")

        # Fetch details for the booking, including user_id
        booking_query = """
            SELECT r.property_id, r.check_in_date, r.check_out_date, r.user_id, p.listing_price
            FROM requests r
            INNER JOIN properties p ON r.property_id = p.property_id
            WHERE r.request_id = %s
        """
        execute_query(booking_query, (request_id,))
        property_id, check_in_date, check_out_date, user_id, listing_price = cursor.fetchone()

        nights_booked = (check_out_date - check_in_date).days
        total_price = nights_booked * listing_price

        # Generate a unique booking ID
        while True:
            booking_id = random.randint(10000, 99999)
            query = "SELECT COUNT(*) FROM bookings WHERE booking_id = %s"
            execute_query(query, (booking_id,))
            if cursor.fetchone()[0] == 0:  # If booking_id is unique
                break

        # Prompt for payment method
        payment_method = input("Enter the payment method (e.g., Credit Card, PayPal): ").strip()

        # Add the booking to the bookings table
        insert_booking_query = """
            INSERT INTO bookings (booking_id, property_id, user_id, check_in_date, check_out_date, 
                                  nights_booked, total_price, payment_method)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        execute_query(insert_booking_query, (booking_id, property_id, user_id, check_in_date, check_out_date, 
                                             nights_booked, total_price, payment_method))
        conn.commit()
        print(f"Booking for Request ID {request_id} has been successfully added to the bookings table.")

        # Update the requests table with the booking ID
        update_request_query = "UPDATE requests SET booking_id = %s WHERE request_id = %s"
        execute_query(update_request_query, (booking_id, request_id))
        conn.commit()
        print(f"Request ID {request_id} updated with Booking ID {booking_id}.")

    except mysql.connector.Error as e:
        print(f"Error during booking insertion: {e}")

In [129]:
def deny_request(request_id):
    """
    Denies a booking request by updating its status to 'Denied'.
    """
    try:
        query = "UPDATE requests SET status = 'Denied' WHERE request_id = %s"
        execute_query(query, (request_id,))
        conn.commit()
        print(f"Request ID {request_id} has been denied.")
    except mysql.connector.Error as e:
        print(f"Error denying the request: {e}")

In [131]:
if __name__ == "__main__":
    try:
        main_menu()
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


Welcome to the Vacation Rental System
1. Owner Login
2. User Login
3. Exit


Enter your choice:  3


Goodbye!
